# Goal

Employee turn-over is a very costly problem for companies. The cost of replacing an employee
if often larger than 100K USD, taking into account the time spent to interview and find a
replacement, placement fees, sign-on bonuses and the loss of productivity for several months.

It is only natural then that data science has started being applied to this area. Understanding
why and when employees are most likely to leave can lead to actions to improve employee
retention as well as planning new hiring in advance. This application of DS is sometimes called
people analytics or people data science (if you see a job title: people data scientist, this is your
job).

In this challenge, you have a data set with info about the employees and have to predict when
employees are going to quit by understanding the main drivers of employee churn.

# Challenge Description

We got employee data from a few companies. We have data about all employees who joined
from 2011/01/24 to 2015/12/13. For each employee, we also know if they are still at the
company as of 2015/12/13 or they have quit. Beside that, we have general info about the
employee, such as avg salary during her tenure, dept, and yrs of experience.

As said above, the goal is to predict employee retention and understand its main drivers.
Specifically, you should:

* Assume, for each company, that the headcount starts from zero on 2011/01/23. Estimate
employee headcount, for each company, on each day, from 2011/01/24 to 2015/12/13.
That is, if by 2012/03/02 2000 people have joined company 1 and 1000 of them have
already quit, then company headcount on 2012/03/02 for company 1 would be 1000.
You should create a table with 3 columns: day, employee_headcount, company_id.

* What are the main factors that drive employee churn? Do they make sense? Explain your
findings.

* If you could add to this data set just one variable that could help explain employee churn,
what would that be?

# Data

## Columns:

* employee_id : id of the employee. Unique by employee per company
* company_id : company id.
* dept : employee dept
* seniority : number of yrs of work experience when hired
* salary: avg yearly salary of the employee during her tenure within the company
* join_date: when the employee joined the company, it can only be between 2011/01/24
and 2015/12/13
* quit_date: when the employee left her job (if she is still employed as of 2015/12/13, this
field is NA)


In [382]:
import pandas as pd
import numpy as np
from natsort import index_natsorted
data = pd.read_csv('employee_retention.csv')
data_origin = data

In [383]:
# look at information of data, we can only quit date have null data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24702 entries, 0 to 24701
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   employee_id  24702 non-null  float64
 1   company_id   24702 non-null  int64  
 2   dept         24702 non-null  object 
 3   seniority    24702 non-null  int64  
 4   salary       24702 non-null  float64
 5   join_date    24702 non-null  object 
 6   quit_date    13510 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 1.3+ MB


In [384]:
data.describe()

,employee_id,company_id,seniority,salary
count,24702.000000,24702.000000,24702.000000,24702.000000
mean,501604.403530,3.426969,14.127803,138183.345478
std,288909.026101,2.700011,8.089520,76058.184573
min,36.000000,1.000000,1.000000,17000.000000
25%,250133.750000,1.000000,7.000000,79000.000000
50%,500793.000000,2.000000,14.000000,123000.000000
75%,753137.250000,5.000000,21.000000,187000.000000
max,999969.000000,12.000000,99.000000,408000.000000


In [385]:
data.head()


,employee_id,company_id,dept,seniority,salary,join_date,quit_date
0,13021.0,7,customer_service,28,89000.0,2014-03-24,2015-10-30
1,825355.0,7,marketing,20,183000.0,2013-04-29,2014-04-04
2,927315.0,4,marketing,14,101000.0,2014-10-13,NaN
3,662910.0,7,customer_service,20,115000.0,2012-05-14,2013-06-07
4,256971.0,2,data_science,23,276000.0,2011-10-17,2014-08-22


In [386]:
data['join_date'] = pd.to_datetime(data.iloc[:, 5])
data['quit_date'] = pd.to_datetime(data.iloc[:, 6])

In [387]:
data['duration'] = data.iloc[:, 6]- data.iloc[:, 5]

In [388]:
data.head()

,employee_id,company_id,dept,seniority,salary,join_date,quit_date,duration
0,13021.0,7,customer_service,28,89000.0,2014-03-24,2015-10-30,585 days
1,825355.0,7,marketing,20,183000.0,2013-04-29,2014-04-04,340 days
2,927315.0,4,marketing,14,101000.0,2014-10-13,NaT,NaT
3,662910.0,7,customer_service,20,115000.0,2012-05-14,2013-06-07,389 days
4,256971.0,2,data_science,23,276000.0,2011-10-17,2014-08-22,1040 days


In [389]:
print("the number of unique employee id", len(data['employee_id'].unique()))
print('the number of all employee id', len(data['employee_id']))


the number of unique employee id 24702
the number of all employee id 24702


In [390]:
# create new table for head_count
head_count = pd.DataFrame()
company = pd.DataFrame()
head_count['day'] = pd.date_range(pd.to_datetime('2011/01/23'), pd.to_datetime('2015/12/13'))
head_count['key'] = 1
company['company'] = data['company_id'].unique()
company['key'] = 1
head_count = head_count.merge(company,on = 'key').drop('key', 1)

D:\temp\ipykernel_57184\253108154.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  head_count = head_count.merge(company,on = 'key').drop('key', 1)


In [391]:
head_count

,day,company
0,2011-01-23,7
1,2011-01-23,4
2,2011-01-23,2
3,2011-01-23,9
4,2011-01-23,1
...,...,...
21427,2015-12-13,5
21428,2015-12-13,3
21429,2015-12-13,8
21430,2015-12-13,11


In [392]:
#calculate how many employee was join in each company for each day
data.set_index('join_date')
data.sort_values(by = 'join_date', key=lambda x: np.argsort(index_natsorted(data["join_date"])),inplace = True)
data['cumcount_join'] = data.groupby('company_id').cumcount()

In [393]:
#calculate how many employee was quit in each company for each day
data.sort_values(by = 'quit_date', inplace = True)
data.reset_index(inplace= True)
data['cumcount_quit'] = data.groupby('company_id').cumcount()

In [394]:
#extract the number of employee was join in each company for each day
join_count = data.groupby(['company_id', 'join_date'], as_index= False).max()
join_count = join_count.iloc[:,[0,1,9]]

In [396]:
#extract the number of employee was quit in each company for each day
quit_count = data.groupby(['company_id', 'quit_date'], as_index= False).max()
quit_count = quit_count.iloc[:,[0,1,10]]

,company_id,quit_date,index,employee_id,dept,seniority,salary,join_date,duration,cumcount_join,cumcount_quit
0,1,2011-10-21,15523,581263.0,sales,5,118000.0,2011-01-24,270 days,16,0
1,1,2011-11-11,3777,71407.0,customer_service,19,107000.0,2011-01-25,290 days,25,1
2,1,2011-11-22,23303,202573.0,engineer,10,199000.0,2011-01-31,295 days,57,2
3,1,2011-11-25,10602,571350.0,marketing,7,119000.0,2011-01-31,298 days,44,3
4,1,2011-12-09,21527,410589.0,engineer,14,208000.0,2011-02-08,304 days,100,4
...,...,...,...,...,...,...,...,...,...,...,...
2719,12,2013-10-18,1680,824438.0,sales,22,109000.0,2011-06-20,851 days,3,6
2720,12,2013-10-25,10255,808516.0,customer_service,3,18000.0,2012-11-12,347 days,7,7
2721,12,2014-02-07,5177,67874.0,sales,5,88000.0,2013-03-11,333 days,11,8
2722,12,2014-09-19,16599,165867.0,customer_service,10,42000.0,2012-11-26,662 days,8,9


In [399]:
#merge join table into head_count by using left join, but there have some null data for join_date
head_count = head_count.merge(join_count, left_on=['day', 'company'], right_on = ['join_date', 'company_id'], how = 'left')

In [400]:
#and then merge quit table into head_count by using left join, but there have some null data for quit_date
head_count = head_count.merge(quit_count, left_on=['day', 'company'], right_on = ['quit_date', 'company_id'], how = 'left')

In [416]:
#look at the null date for head count table, find join date and quit date have many null data, 
# we should set the number of employee on first day equal to zero
# use the number of join subtract the number of quit is equal head count
head_count.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21432 entries, 0 to 21431
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   day            21432 non-null  datetime64[ns]
 1   company        21432 non-null  int64         
 2   company_id_x   5130 non-null   float64       
 3   join_date      5130 non-null   datetime64[ns]
 4   cumcount_join  5130 non-null   float64       
 5   company_id_y   2724 non-null   float64       
 6   quit_date      2724 non-null   datetime64[ns]
 7   cumcount_quit  2724 non-null   float64       
dtypes: datetime64[ns](3), float64(4), int64(1)
memory usage: 1.5 MB


In [403]:
# so if the join number or quit number is null, set the join number or quit number equal to the join number or quit number of previous day 
head_count_new = pd.DataFrame()
for j in range(1, 13):
    head_count_company = head_count[head_count['company'] == j]
    head_count_company.iloc[0, 4] = 0
    head_count_company.iloc[0, 7] = 0
    for i in range(1, 1786):
        if pd.isnull(head_count_company.iloc[i,4]):
            head_count_company.iloc[i, 4] = head_count_company.iloc[i-1,4]
        else:
            head_count_company.iloc[i, 4] = head_count_company.iloc[i, 4]
        if pd.isnull(head_count_company.iloc[i,7]):
            head_count_company.iloc[i, 7] = head_count_company.iloc[i-1,7]
        else:
            head_count_company.iloc[i, 7] = head_count_company.iloc[i, 7]
    head_count_new = pd.concat([head_count_new, head_count_company])

In [410]:
# calculate the head count of each company for each day
head_count_new['head_count'] = head_count_new['cumcount_join'] -head_count_new['cumcount_quit']

In [413]:
head_count_table = head_count_new.iloc[:,[0,1,8]]

In [415]:
head_count_table.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21432 entries, 4 to 21431
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   day         21432 non-null  datetime64[ns]
 1   company     21432 non-null  int64         
 2   head_count  21432 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 669.8 KB
